<a href="https://colab.research.google.com/github/rimgosu/ColabBackup/blob/master/ex04_%EA%B0%9C_%EA%B3%A0%EC%96%91%EC%9D%B4_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# /content/drive/MyDrive/Colab Notebooks/DeepLearning(Spring)/data/dogs_vs_cats_small
train_dir = '/content/drive/MyDrive/Colab Notebooks/DeepLearning(Spring)/data/dogs_vs_cats_small/train'
valid_dir = '/content/drive/MyDrive/Colab Notebooks/DeepLearning(Spring)/data/dogs_vs_cats_small/validation'

In [2]:
# 하나의 변수에 이미지 파일 전부다 합치기
# 픽셀값 변경 (0~255 / 정수) > (0~1 / 실수) : 정규화
## 1. 숫자 크기 줄이기 > 연산량 감소
## 2. 분산 줄이기 > 연산의 오류 줄어듦
# 이미지 크기 맞춰주기 (150, 150)
# 라벨링
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# 픽셀값 변경하기
## 프로그래밍에서 연산상에서 타입이 변경되도록 하기 (묵시적 형변환 int => float)
generator = ImageDataGenerator(rescale= 1./255)



In [4]:
# 하나의 변수에 이미지 파일 전부 다 합치기
# 이미지 크기 동일하게 만들어주지
# 라벨링
train_generator = generator.flow_from_directory(
    directory = train_dir,
    target_size = (150,150), # 변환할 이미지의 크기
    batch_size = 100, # 한 번에 변환할 이미지 갯수
    class_mode = 'binary' # 라벨링 방법, 다중분류 : categorical
)

Found 2000 images belonging to 2 classes.


In [5]:
# 하나의 변수에 이미지 파일 전부 다 합치기
# 이미지 크기 동일하게 만들어주지
# 라벨링
valid_generator = generator.flow_from_directory(
    directory = valid_dir,
    target_size = (150,150), # 변환할 이미지의 크기
    batch_size = 100, # 한 번에 변환할 이미지 갯수
    class_mode = 'binary' # 라벨링 방법, 다중분류 : categorical
)

Found 1000 images belonging to 2 classes.


In [6]:
# CNN 모델 설계
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [8]:
model1 = Sequential()

model1.add(Conv2D(
    filters = 32,
    kernel_size = (3,3),
    input_shape = (150,150,3),
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size= (2,2)
))

model1.add(Conv2D(
    filters = 32,
    kernel_size = (3,3),
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size= (2,2)
))

In [11]:
model1.add(Flatten())
model1.add(Dense(units = 32, activation = 'relu'))
model1.add(Dense(units= 1, activation = 'sigmoid'))

In [12]:
model1.compile(
    loss = 'binary_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)

In [13]:
model1.fit(
    train_generator,
    epochs = 20,
    validation_data = valid_generator
)

Epoch 1/20
20/20 [==============================] - 747s 37s/step - loss: 0.8069 - accuracy: 0.5040 - val_loss: 0.6760 - val_accuracy: 0.5830
Epoch 2/20
20/20 [==============================] - 9s 451ms/step - loss: 0.6613 - accuracy: 0.6005 - val_loss: 0.6602 - val_accuracy: 0.5900
Epoch 3/20
20/20 [==============================] - 11s 543ms/step - loss: 0.6317 - accuracy: 0.6320 - val_loss: 0.6296 - val_accuracy: 0.6150
Epoch 4/20
20/20 [==============================] - 8s 405ms/step - loss: 0.5659 - accuracy: 0.7100 - val_loss: 0.6255 - val_accuracy: 0.6350
Epoch 5/20
20/20 [==============================] - 9s 443ms/step - loss: 0.5197 - accuracy: 0.7540 - val_loss: 0.5765 - val_accuracy: 0.7060
Epoch 6/20
20/20 [==============================] - 11s 557ms/step - loss: 0.4778 - accuracy: 0.7790 - val_loss: 0.6034 - val_accuracy: 0.6860
Epoch 7/20
20/20 [==============================] - 11s 536ms/step - loss: 0.4365 - accuracy: 0.8085 - val_loss: 0.5792 - val_accuracy: 0.6950
Epo